In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('fake-news/train.csv')

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


In [4]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [5]:
df = df.dropna()

In [6]:
df.isnull().sum()

id        0
title     0
author    0
text      0
label     0
dtype: int64

In [7]:
X = df.drop('label', axis=1)

In [8]:
y = df['label']

In [9]:
X.shape

(18285, 4)

In [10]:
y.shape

(18285,)

In [11]:
import tensorflow as tf

In [12]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense

In [13]:
voc_size = 5000

In [14]:
messages = X.copy()

In [15]:
messages['title'][1]

'FLYNN: Hillary Clinton, Big Woman on Campus - Breitbart'

In [18]:
import nltk
import re
from nltk.corpus import stopwords

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to C:\Users\Payal
[nltk_data]     Arora\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [31]:
### Dataset Preprocessing
from nltk.stem.porter import PorterStemmer ##stemming purpose
ps = PorterStemmer()
corpus = []
for i in range(0, len(messages)):
    review = re.sub('[^a-zA-Z]', ' ', messages['title'][i])
    review = review.lower()
    review = review.split()

    review = [ps.stem(word) for word in review if word not in stopwords.words('english')]
    review = ' '.join(review)
    corpus.append(review)

In [32]:
corpus

['hous dem aid even see comey letter jason chaffetz tweet',
 'flynn hillari clinton big woman campu breitbart',
 'truth might get fire',
 'civilian kill singl us airstrik identifi',
 'iranian woman jail fiction unpublish stori woman stone death adulteri',
 'jacki mason hollywood would love trump bomb north korea lack tran bathroom exclus video breitbart',
 'beno hamon win french socialist parti presidenti nomin new york time',
 'back channel plan ukrain russia courtesi trump associ new york time',
 'obama organ action partner soro link indivis disrupt trump agenda',
 'bbc comedi sketch real housew isi caus outrag',
 'russian research discov secret nazi militari base treasur hunter arctic photo',
 'us offici see link trump russia',
 'ye paid govern troll social media blog forum websit',
 'major leagu soccer argentin find home success new york time',
 'well fargo chief abruptli step new york time',
 'anonym donor pay million releas everyon arrest dakota access pipelin',
 'fbi close hilla

In [34]:
onehot_repr = [one_hot(words,voc_size) for words in corpus]
print(onehot_repr)

[[2734, 4339, 4421, 3293, 989, 256, 11, 3362, 2360, 4090], [4718, 4337, 2802, 899, 3780, 322, 4388], [675, 1555, 2735, 4638], [2217, 2472, 2904, 423, 2800, 1971], [1461, 3780, 1353, 175, 3251, 4383, 3780, 594, 4446, 4844], [1376, 1040, 4062, 1645, 1315, 859, 4728, 2209, 1101, 215, 1253, 2022, 2529, 3108, 4388], [4465, 445, 1038, 36, 3908, 4690, 607, 4704, 1398, 2279, 3667], [743, 3466, 4320, 2141, 4354, 2972, 859, 1726, 1398, 2279, 3667], [3111, 1982, 2352, 2880, 1278, 2412, 3643, 1912, 859, 736], [1222, 4287, 861, 4416, 1570, 2152, 236, 2863], [1674, 4399, 2230, 4043, 3570, 3917, 4543, 1041, 4187, 3685, 4276], [423, 371, 989, 2412, 859, 4354], [3546, 3480, 2236, 1082, 1537, 3212, 4437, 3979, 3325], [1161, 1777, 2128, 2361, 1202, 2783, 1250, 1398, 2279, 3667], [4345, 4720, 108, 478, 2715, 1398, 2279, 3667], [2777, 2579, 906, 2320, 4928, 3303, 1068, 1857, 780, 3137], [4768, 1760, 4337], [2142, 2283, 1024, 1017, 859, 4164, 2007, 4388], [4527, 78, 2802, 208, 4282, 1366, 3861, 1621, 4383],

In [35]:
corpus[1]

'flynn hillari clinton big woman campu breitbart'

In [36]:
onehot_repr[1]

[4718, 4337, 2802, 899, 3780, 322, 4388]

In [37]:
sent_length = 20
embedded_docs = pad_sequences(onehot_repr, padding='pre', maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ... 3362 2360 4090]
 [   0    0    0 ... 3780  322 4388]
 [   0    0    0 ... 1555 2735 4638]
 ...
 [   0    0    0 ... 1398 2279 3667]
 [   0    0    0 ...  604 3565 1829]
 [   0    0    0 ... 2793  872 3867]]


In [38]:
embedding_vector_features = 40
model=Sequential()
model.add(Embedding(voc_size, embedding_vector_features, input_length = sent_length))
model.add(LSTM(100))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', optimizer='adam', metrics = ['accuracy'])
print(model.summary())

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 20, 40)            200000    
                                                                 
 lstm (LSTM)                 (None, 100)               56400     
                                                                 
 dense (Dense)               (None, 1)                 101       
                                                                 
Total params: 256501 (1001.96 KB)
Trainable params: 256501 (1001.96 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________
None


In [40]:
import numpy as np
X_final = np.array(embedded_docs)
y_final = np.array(y)

In [41]:
X_final.shape, y_final.shape

((18285, 20), (18285,))

In [42]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.20, random_state=42)

In [44]:
model.fit(X_train,y_train, validation_data= (X_test,y_test), epochs=10, batch_size=64)

Epoch 1/10
229/229 [==============================] - 17s 23ms/step - loss: 0.3222 - accuracy: 0.8419 - val_loss: 0.1944 - val_accuracy: 0.9128
Epoch 2/10
229/229 [==============================] - 4s 19ms/step - loss: 0.1392 - accuracy: 0.9458 - val_loss: 0.2001 - val_accuracy: 0.9174
Epoch 3/10
229/229 [==============================] - 5s 22ms/step - loss: 0.1017 - accuracy: 0.9627 - val_loss: 0.2262 - val_accuracy: 0.9114
Epoch 4/10
229/229 [==============================] - 5s 23ms/step - loss: 0.0684 - accuracy: 0.9765 - val_loss: 0.2847 - val_accuracy: 0.9125
Epoch 5/10
229/229 [==============================] - 5s 23ms/step - loss: 0.0463 - accuracy: 0.9843 - val_loss: 0.3179 - val_accuracy: 0.9092
Epoch 6/10
229/229 [==============================] - 5s 22ms/step - loss: 0.0290 - accuracy: 0.9916 - val_loss: 0.3689 - val_accuracy: 0.9100
Epoch 7/10
229/229 [==============================] - 5s 23ms/step - loss: 0.0187 - accuracy: 0.9939 - val_loss: 0.4681 - val_accuracy: 0.911

In [45]:
y_pred = model.predict(X_test)

115/115 [==============================] - 1s 5ms/step


In [46]:
y_pred = np.where(y_pred > 0.5, 1, 0)

In [49]:
from sklearn.metrics import accuracy_score
score = accuracy_score(y_test,y_pred)

In [50]:
print(score)

0.9135903746240087


In [54]:
result = 'Fake News' if y_pred[2][0] >= 0.5 else 'Real News'
print("Prediction:", result)

Prediction: Real News
